In [137]:
import pandas as pd
import numpy as np

In [138]:
df = pd.read_excel('azure-export-2024-08-01-00-01-38.xlsx')

In [139]:
df.shape

(20593, 37)

In [140]:
df.columns

Index(['Name', 'Package Picture', 'URL',
       'P3, Cash Discount, Installment/month', 'Installment Limit',
       'Price to Reserve for This Package', 'Shop Name', 'Category',
       'Category Tags', 'Preview 1-10', 'Selling Point', 'Meta Keywords',
       'Brand', 'Min/Max Age',
       'Locations, Time Open/Close, How to Transport, Parking, Google Maps',
       'Meta Description', 'Price Details', 'Package Details',
       'Important Info', 'Payment Booking Info', 'General Info',
       'Early Signs for Diagnosis', 'How to Diagnose', 'HDCare Summary',
       'Common Question', 'Know This Disease', 'Courses of Action',
       'Signals to Proceed Surgery', 'Get to Know This Surgery', 'Comparisons',
       'Getting Ready', 'Recovery', 'Side Effects', 'Review 4-5 Stars',
       'Brand Option in Thai Name', 'Brand Ranking (Position)', 'FAQ'],
      dtype='object')

In [141]:
df['P3, Cash Discount, Installment/month'][0]

'P3: 969 THB, Cash: 969.0 THB, Installment/month: N/A'

In [142]:
extracted_cols = df['P3, Cash Discount, Installment/month'].str.extract(r'P3: (\d*) THB, Cash: (\.|\d)* THB, Installment/month: (.*)')
extracted_cols.columns = ['P3', 'Cash Discount', 'Installment/month']

# Converting the columns to numeric types
extracted_cols['P3'] = pd.to_numeric(extracted_cols['P3'])
extracted_cols['Cash Discount'] = pd.to_numeric(extracted_cols['Cash Discount'])

# Adding the new column "price after cash discount"
extracted_cols['Price After Cash Discount'] = extracted_cols['P3'] - extracted_cols['Cash Discount']

extracted_cols

,P3,Cash Discount,Installment/month,Price After Cash Discount
0,969,0,N/A,969
1,4455,0,N/A,4455
2,989,0,N/A,989
3,4455,0,N/A,4455
4,4365,0,N/A,4365
...,...,...,...,...
20588,8549,0,N/A,8549
20589,3799,0,N/A,3799
20590,44555,0,N/A,44555
20591,48355,0,N/A,48355


In [143]:
df.drop(columns=['Category Tags', 'Preview 1-10', 'Selling Point', 'Meta Keywords', 'Meta Description', 'Payment Booking Info', 'Brand Option in Thai Name', 'Brand Ranking (Position)'], inplace=True)

In [144]:
for i, col in enumerate(extracted_cols.columns):
    df.insert(df.columns.get_loc('P3, Cash Discount, Installment/month') + 1 + i, col, extracted_cols[col])

df.drop(columns=['P3, Cash Discount, Installment/month'], inplace=True)

df['Installment/month'] = np.where(df['Installment/month'] == 'N/A', '', df['Installment/month'])

df.columns

Index(['Name', 'Package Picture', 'URL', 'P3', 'Cash Discount',
       'Installment/month', 'Price After Cash Discount', 'Installment Limit',
       'Price to Reserve for This Package', 'Shop Name', 'Category', 'Brand',
       'Min/Max Age',
       'Locations, Time Open/Close, How to Transport, Parking, Google Maps',
       'Price Details', 'Package Details', 'Important Info', 'General Info',
       'Early Signs for Diagnosis', 'How to Diagnose', 'HDCare Summary',
       'Common Question', 'Know This Disease', 'Courses of Action',
       'Signals to Proceed Surgery', 'Get to Know This Surgery', 'Comparisons',
       'Getting Ready', 'Recovery', 'Side Effects', 'Review 4-5 Stars', 'FAQ'],
      dtype='object')

In [145]:
# Assuming your DataFrame is named df
nan_counts = df.isna().sum()

# Display the counts
print(nan_counts)

Name                                                                      0
Package Picture                                                         739
URL                                                                       0
P3                                                                        0
Cash Discount                                                             0
Installment/month                                                         0
Price After Cash Discount                                                 0
Installment Limit                                                     20461
Price to Reserve for This Package                                     17290
Shop Name                                                                 0
Category                                                                  0
Brand                                                                     0
Min/Max Age                                                           20538
Locations, T

In [146]:
import re

def clean_location_info(df, column_name):

  patterns = [
      r"When to open:.+\n",
      r"Parking lot:.+\n",
      r'How to transport:.+\n',
      r'Google Maps link: ',
  ]
  combined_pattern = "|".join(patterns)

  # Handle non-string values before applying regex
  def clean_value(x):
    # Convert non-strings to strings (if possible)
    if not pd.api.types.is_string_dtype(x):
      x = str(x)  # Try converting to string
    # Handle missing values (e.g., NaN)
    if pd.isna(x):
      x = ""  # Replace with an empty string for missing values
    return re.sub(combined_pattern, '', x)

  df[column_name] = df[column_name].apply(clean_value)
  return df

# Example usage
cleaned_df = clean_location_info(df.copy(), "Locations, Time Open/Close, How to Transport, Parking, Google Maps")


In [147]:
# Define the new column names in the order they should appear
new_column_names = [
    'package_name',
    'package_picture',
    'url',
    'price',
    'cash_discount',
    'installment_month',
    'price_after_cash_discount',
    'installment_limit',
    'price_to_reserve_for_this_package',
    'shop_name',
    'category',
    'brand',
    'min_max_age',
    'locations',
    'price_details',
    'package_details',
    'important_info',
    'general_info',
    'early_signs_for_diagnosis',
    'how_to_diagnose',
    'hdcare_summary',
    'common_question',
    'know_this_disease',
    'courses_of_action',
    'signals_to_proceed_surgery',
    'get_to_know_this_surgery',
    'comparisons',
    'getting_ready',
    'recovery',
    'side_effects',
    'review_4_5_stars',
    'faq',
]


# Generate the column mapping
current_columns = cleaned_df.columns.tolist()
if len(current_columns) != len(new_column_names):
    raise ValueError("The number of columns in the CSV does not match the number of new column names")

column_mapping = {current: new for current, new in zip(current_columns, new_column_names)}

# Rename columns
cleaned_df.rename(columns=column_mapping, inplace=True)

cleaned_df.columns


Index(['package_name', 'package_picture', 'url', 'price', 'cash_discount',
       'installment_month', 'price_after_cash_discount', 'installment_limit',
       'price_to_reserve_for_this_package', 'shop_name', 'category', 'brand',
       'min_max_age', 'locations', 'price_details', 'package_details',
       'important_info', 'general_info', 'early_signs_for_diagnosis',
       'how_to_diagnose', 'hdcare_summary', 'common_question',
       'know_this_disease', 'courses_of_action', 'signals_to_proceed_surgery',
       'get_to_know_this_surgery', 'comparisons', 'getting_ready', 'recovery',
       'side_effects', 'review_4_5_stars', 'faq'],
      dtype='object')

In [148]:
numerical_cols = cleaned_df.select_dtypes(include=['number']).columns
cleaned_df[numerical_cols] = cleaned_df[numerical_cols].fillna(np.nan)

# Replace price 0 with Nan
cleaned_df['price'] = cleaned_df['price'].replace(0, np.nan)

# For categorical columns, fill missing values with an empty string
categorical_cols = cleaned_df.select_dtypes(include=['object']).columns
cleaned_df[categorical_cols] = cleaned_df[categorical_cols].fillna("")


In [149]:
def to_csv(df, csv_path, start_row, end_row):
    df.iloc[start_row:end_row].to_csv(csv_path, index=False,)
    print(f"{csv_path} created successfully!")
    return csv_path

In [150]:
test_df = cleaned_df[:100]

In [151]:
to_csv(cleaned_df, 'packages.csv', 0, None)

packages.csv created successfully!


'packages.csv'